# __Datenbereinigung Teil 2: Bereinigung der Anforderungsdaten__

In diesem Schritt werden die Daten bereinigt, welche für das Wörterbuch mit den häufigst genannten Anforderungen benötigt werden.

In [2]:
# Import der benötigten Libraries
from time import time
import pandas as pd
import nltk
from nltk.corpus import stopwords 
from bs4 import BeautifulSoup
from HanTa import HanoverTagger as ht
import string
from nltk.tokenize.treebank import TreebankWordDetokenizer
import os

# Working-Directory
os.chdir('C:/Users/Sven Konermann/Documents/Master/2_Semester/aF_E_Faelle/Unterlagen/JobCloud_Daten')

In [3]:
# Einlesen der Daten über ein csv File
data = pd.read_csv("Inserate_Anforderungsanalyse_AF_E.csv", sep=";")
print(data.shape)
data.info()

(75980, 14)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75980 entries, 0 to 75979
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Unnamed: 0             75980 non-null  int64 
 1   ad_id                  75980 non-null  object
 2   active_start_on        75980 non-null  object
 3   active_end_on          75980 non-null  object
 4   title                  75980 non-null  object
 5   text                   75980 non-null  object
 6   branchen_name          75539 non-null  object
 7   customer_branche_name  75978 non-null  object
 8   lvl1_region_name       75950 non-null  object
 9   region_name            75950 non-null  object
 10  lvl1_jobcategory_name  75965 non-null  object
 11  jobcategory_name       75965 non-null  object
 12  duration               75980 non-null  int64 
 13  year                   75980 non-null  int64 
dtypes: int64(3), object(11)
memory usage: 8.1+ MB


In [4]:
# Listenelemente in einer neuen Spalte abspeichern
start = time()

column = []
for i in data['text']:
    text=''
    soup = BeautifulSoup(i, 'lxml')
    tags = soup.find_all('li')
    #print(tags)
    for t in tags:
        text = text + ' ' + t.text 
    column.append(text)

data['Text_Anforderungen'] = column

print(f'Benötigte Zeit: {time() - start} Sekunden')

Benötigte Zeit: 71.63456702232361 Sekunden


In [5]:
# Lemmatisierung der Anforderungen
start = time()
tagger = ht.HanoverTagger('morphmodel_ger.pgz')

for mail in data['Text_Anforderungen']:
    lemma = [lemma for (word,lemma,pos) in tagger.tag_sent(mail.split())]
    index = data[data['Text_Anforderungen']==mail].index
    data.loc[index, 'Text_Anforderungen'] = (' '.join(lemma))
    
print(f'Benötigte Zeit: {time() - start} Sekunden')

Benötigte Zeit: 8474.594832658768 Sekunden


In [6]:
# Text in Kleinbuchstaben umwandeln
data['Text_Anforderungen'] = data['Text_Anforderungen'].str.lower()

In [7]:
# Satzzeichen löschen
def remove_punctuation(txt):
    txt_nopunct = "".join([c for c in txt if c not in string.punctuation])
    return txt_nopunct

data['Text_Anforderungen'] = data['Text_Anforderungen'].apply(lambda x: remove_punctuation(x))

In [8]:
# Tabs \t aus den Texten entfernen
data['Text_Anforderungen'] = data['Text_Anforderungen'].str.replace("\t", "", regex=True)
data['Text_Anforderungen']

0        teil in verschieden softwareapplikation erfass...
1        warenannahme kennzeichnung und einlagerung der...
2        termin und reiseplanung inkl flugbuchung und s...
3        vertretung des abteilungsleiter bei sein abwes...
4        erarbeiten von business analyse für unser soft...
                               ...                        
75975    du sein verantwortlich für die schadenregulier...
75976    erhebung der anamnese beurteilung der medizini...
75977    diverser administrativ tätigkeit z b pflege de...
75978    selbständig führung der finanz betriebs und an...
75979    spenglerarbeit auf steil und flachdächern werk...
Name: Text_Anforderungen, Length: 75980, dtype: object

In [9]:
# Ähnliche Anforderungen manuell mit einem Hauptbegriff ersetzen
data['Text_Anforderungen'] = data['Text_Anforderungen'].str.replace("teamplayer", "teamfähig", regex=True)
data['Text_Anforderungen'] = data['Text_Anforderungen'].str.replace("verhandlungssicher", "verhandlungsgeschick", regex=True)
data['Text_Anforderungen'] = data['Text_Anforderungen'].str.replace("kundenorientierte", "kundenorientierung", regex=True)
data['Text_Anforderungen'] = data['Text_Anforderungen'].str.replace("projekt management", "projektleitungserfahrung", regex=True)
data['Text_Anforderungen'] = data['Text_Anforderungen'].str.replace("projektmanagement", "projektleitungserfahrung", regex=True)
data['Text_Anforderungen'] = data['Text_Anforderungen'].str.replace("zusammenarbeit", "teamfähig", regex=True)
data['Text_Anforderungen'] = data['Text_Anforderungen'].str.replace("kommunikation", "kommunikationsfähigkeit", regex=True)
data['Text_Anforderungen'] = data['Text_Anforderungen'].str.replace("kommunikationsfähigkeiten", "kommunikationsfähigkeit", regex=True)
data['Text_Anforderungen'] = data['Text_Anforderungen'].str.replace("dwh", "data warehouse", regex=True)
data['Text_Anforderungen'] = data['Text_Anforderungen'].str.replace("dienstleistungsorientierung", "dienstleistungsorientiert", regex=True)
data['Text_Anforderungen'] = data['Text_Anforderungen'].str.replace("eigenverantwortung", "eigenverantwortlich", regex=True)

In [10]:
# Word-Tokenization 
#nltk.download('punkt')

start = time()
data['text_tok'] = data.apply(lambda row: nltk.word_tokenize(row['Text_Anforderungen']), axis=1)
print(f'Benötigte Zeit: {time() - start} Sekunden')

Benötigte Zeit: 34.05356812477112 Sekunden


In [11]:
# Stopwörter löschen
nltk.download('stopwords')

stop_words = set(stopwords.words('german'))
data['text_tok'] = data['text_tok'].apply(lambda x: [item for item in x if item not in stop_words])

[nltk_data] Downloading package stopwords to C:\Users\Sven
[nltk_data]     Konermann\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [12]:
# Tokenizing rückgängig machen, da die Tokenizierung nicht in eine csv-Datei überführt werden kann
column = []
for i in data['text_tok']:
    test = TreebankWordDetokenizer().detokenize(i)
    column.append(test)

data['Text_Anforderungen'] = column

In [13]:
# Löschen der tokenisierten Spalte
data = data.drop(['text_tok'], axis = 1)

In [14]:
# Erneutes Löschen der Duplikate, da sich die Texte verändert haben
data = data.drop_duplicates(subset='Text_Anforderungen', keep="last")

In [15]:
data.to_csv("Inserate_clean_Anforderungsanalyse_AF_E.csv", sep=";")

nächster Schritt: Wörterbuch erstellen (Analyse_der_Anforderungen.ipynb)